In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-02 21:36:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2023-03-02 21:36:26 (10.1 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

# Load in employee.csv from S3 into a DataFrame
# url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_3/employee.csv"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

2402458

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-08-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-08-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-08-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id", "product_title"])
products_df.dropDuplicates()
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000SAQCWC|The Cravings Plac...|
|B00509LVIQ|Mauna Loa Macadam...|
|B00KHXESLC|Organic Matcha Gr...|
|B000F8JIIC|15oz Raspberry Ly...|
|B004ZWR9RQ|Stride Spark Kine...|
|B00AL6QBZ6|Herr's Popcorn Ho...|
|B00BCNSTRA|Larabar uber, 1.4...|
|B0089MM2BG|Shirakiku Soba No...|
|B00Y1C9770|Jif Chocolate Nut...|
|B00J074W94|Orgain Organic Pl...|
|B0001VKKOO|Bragg - All Natur...|
|B007TGH4CK|Wholesome Sweeten...|
|B002HMN6SC|Kadoya Pure Sesam...|
|B004NRHAZO|Nishiki Premium B...|
|B00V7LJIG8|Everly Passion Fr...|
|B0000E2YFI|Charms Blue Razzb...|
|B00VK5SQOQ|Food Should Taste...|
|B00XDXMLL2|Skippy Creamy Pea...|
|B00MN4KX3A|Celestial Seasoni...|
|B004NTCE1M|Nutiva Organic Vi...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})\

customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   21294263|                 1|
|    8619125|                 1|
|   43626894|                 1|
|   38209321|                 4|
|   40568205|                 1|
|   13480593|                 1|
|   19505799|                 1|
|   43150161|                 2|
|   28802315|                 2|
|   35329257|                 1|
|   49101505|                 1|
|    1117644|                 1|
|   28377689|                 1|
|   23493243|                 1|
|   48670265|                 1|
|   49041516|                 3|
|   37499049|                 2|
|   10920752|                 1|
|    5627998|                 1|
|     135423|                 1|
+-----------+------------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_db"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
